# Oct 26, 2024: Membership profiles per roi across modes

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
from scipy.spatial.distance import cdist
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from scipy.spatial.distance import jensenshannon
from multiprocessing import Pool
import glob
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import DBSCAN

import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations
from collections import Counter
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 25
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{PARC_DESC}_desc-labels.txt')

In [6]:
def get_dist_mat(X):
    # Jenson Shannon divergence between two probability mass functions
    D = np.zeros((X.shape[0], X.shape[0]))
    for i, j in combinations(range(X.shape[0]), 2):
        D[i, j] = jensenshannon(X[i, :], X[j, :])
        D[j, i] = D[i, j]
    return D

def find_clusters(args, X):
    D = get_dist_mat(X)
    clustering = DBSCAN(eps=args.eps, min_samples=1)
    grps = clustering.fit_predict(D)
    return D, grps

def get_unique_profiles_and_strengths_per_roi(args, sub, roi, X, omegas, level=-1):
    D, grps = find_clusters(args, X)

    U = []
    S = []
    for g in np.unique(grps):
        U += [np.round(np.mean(X[grps == g, :], axis=0), decimals=3)]
        S += [np.sum(omegas[grps == g])]
    return pd.DataFrame(dict(
        sub=[sub]*len(S),
        level=[level]*len(S) if level != -1 else None,
        roi=[roi]*len(S),
        profile=U,
        strength=S
    )).sort_values(by=['strength'], ascending=False)

def get_membership_profiles_per_sub(args, df, level=-1):
    sub = df.iloc[0]['sub']
    omegas = df['omega'].to_numpy()

    num_modes = len(df)
    num_comms = np.max(df['pi'].apply(lambda pi: pi.shape[-1]))
    num_rois = df.iloc[0]['pi'].shape[0]
    M = np.zeros((num_rois, num_modes, num_comms)) # membership profile matrix

    for idx_mode in range(num_modes):
        pi = df[df['mode_id'] == idx_mode]['pi'].to_list()[0]
        M[:, idx_mode, :pi.shape[-1]] = pi

    profiles_sub_df = []
    for roi in (range(M.shape[0])):
        X = M[roi, :, :]
        prow = get_unique_profiles_and_strengths_per_roi(args, sub, roi, X, omegas, level=level)
        profiles_sub_df += [prow]
    profiles_sub_df = pd.concat(profiles_sub_df).reset_index(drop=True)
    return profiles_sub_df

def get_membership_profiles(args, indiv_files):
    profiles_df = []
    for indiv_file in tqdm(indiv_files):
        with open(indiv_file, 'rb') as f:
            indiv_marginals_df = pickle.load(f)

        # sub = [s for s in indiv_file.split('/') if 'sub' in s][0].split('-')[-1]
        indiv_marginals_df['pi'] = indiv_marginals_df['pi'].apply(lambda pi: np.round(pi, decimals=3))

        if args.sbm in ['a', 'd']:
            profiles_sub_df = get_membership_profiles_per_sub(args, df=indiv_marginals_df)
            profiles_df += [profiles_sub_df]
        if args.sbm in ['h']:
            for level in indiv_marginals_df['level'].unique():
                profiles_sub_df = get_membership_profiles_per_sub(args, df=indiv_marginals_df[indiv_marginals_df['level'] == level], level=level)
                profiles_df += [profiles_sub_df]
    return pd.concat(profiles_df).reset_index(drop=True)

In [ ]:
def get_group_level_num_unique_mem_profiles(args, profiles_df, level=-1):
    rois = profiles_df['roi'].unique()
    num_unique_profiles = []
    for roi in rois:
        num_unique_profiles += [profiles_df[profiles_df['roi'] == roi]['sub'].value_counts().sort_index().mode().to_list()]

    max_num_unique = np.max([item for sublist in num_unique_profiles for item in sublist])
    MLB = MultiLabelBinarizer(classes=list(range(max_num_unique+1))).fit_transform(num_unique_profiles)

    fig, axs = plt.subplots(1, 1, figsize=(4, 6), dpi=90)
    ax = axs
    sns.heatmap(MLB, ax=ax, cmap=cc.cm.CET_L12)
    title = f'{SBM}' if level == -1 else f'{SBM}_level-{level}'
    ax.set(xlabel=f'#unique profiles', ylabel=f'roi', title=title)

    return MLB, fig

In [8]:
def max_prob_profile_match(args, profiles_df):
    max_idxs = profiles_df.groupby(by=['sub', 'roi'])['strength'].idxmax()
    maxp_df = profiles_df.loc[max_idxs]
    # maxp_df

    matches = []
    for key, group in maxp_df.groupby(by=['roi']):
        roi = key[0]
        num_comms = np.max(group['profile'].apply(lambda p: len(p)))
        num_subs = len(group)
        X = np.zeros((num_subs, num_comms))
        for idx, profile in enumerate(group['profile']):
            X[idx, :len(profile)] = profile
        
        D, grps = find_clusters(args, X)
        grp_lens = np.array(list(Counter(grps).values()))
        match = np.max(grp_lens / np.sum(grp_lens))
        matches += [(roi, match)]
    max_matches_df = pd.DataFrame(matches, columns=['roi', 'match'])
    return max_matches_df['match'].to_numpy()[:, None]

In [9]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def profiles_to_nifti(args, X, folder, name='num-profiles'):
    os.system(f'mkdir -p {folder}')
    in_files = []
    for idx_num in range(X.shape[-1]):
        x = X[:, idx_num]
        x_img = np.zeros_like(parcels)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels == roi) * (x[idx])
        
        file = f'{folder}/num-{idx_num:02d}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    out_file = f'{folder}/desc-{name}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [10]:
sbms_list = [
    (True, 'a'),
    (True, 'd'),
    (True, 'h'),
    (False, 'd'),
    (False, 'h'),
]

In [ ]:
for sbm in sbms_list[:]:
    args.dc, args.sbm = sbm

    args.nested = args.sbm == 'h'

    args.force_niter = 40000
    args.num_draws = int((1/2) * args.force_niter)

    def sbm_name(args):
        dc = f'dc' if args.dc else f'nd'
        dc = f'' if args.sbm in ['a'] else dc
        file = f'sbm-{dc}-{args.sbm}'
        return file

    SBM = sbm_name(args)
    print(SBM)

    indiv_files = sorted(glob.glob(
        f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-marginals-df.pkl', 
        recursive=True
    ))
    display(indiv_files)

    args.eps = 0.35
    if args.sbm in ['a', 'd']:
        profiles_df = get_membership_profiles(args, indiv_files)
        out_folder = f'{ESTIM_path}/group/membership-profiles/{SBM}'
        os.system(f'mkdir -p {out_folder}')

        # NUMPs : Number of Unique Membership Profile s
        NUMPs, fig = get_group_level_num_unique_mem_profiles(args, profiles_df)
        profiles_to_nifti(args, NUMPs, out_folder)

        X = max_prob_profile_match(args, profiles_df)
        profiles_to_nifti(args, X, folder=out_folder, name='max-profile-match')

    if args.sbm in ['h']:
        profiles_df = get_membership_profiles(args, indiv_files)
        for level in profiles_df['level'].unique():
            df = profiles_df[profiles_df['level'] == level]
            out_folder = f'{ESTIM_path}/group/membership-profiles/{SBM}/level-{level}'
            os.system(f'mkdir -p {out_folder}')

            NUMPs, fig = get_group_level_num_unique_mem_profiles(args, df, level=level)
            profiles_to_nifti(args, NUMPs, out_folder)
            fig.savefig(f'{out_folder}/desc-num-profiles.pdf', bbox_inches='tight')

            X = max_prob_profile_match(args, df)
            profiles_to_nifti(args, X, out_folder, name='max-profile-match')

    plt.close('all')
    # break

sbm--a


['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/gr

100%|██████████| 10/10 [00:04<00:00,  2.19it/s]


241028-16:29:25,83 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.083346:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:29:25,93 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.093241:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:29:25,96 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.096004:*+ WARNING: Set TR of output dataset to 1.0 s
241028-16:29:25,190 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.190127:++ elapsed time = 0.1 s
241028-16:29:25,867 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.867276:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:29:25,870 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.870359:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:29:25,904 nipype.interface INFO:
	 stderr 2024-10-28T16:29:25.904341:++ elapsed time = 0.0 s
sbm-dc-d


['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-

...

100%|██████████| 10/10 [00:03<00:00,  2.98it/s]


241028-16:29:57,991 nipype.interface INFO:
	 stderr 2024-10-28T16:29:57.991052:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:29:57,997 nipype.interface INFO:
	 stderr 2024-10-28T16:29:57.997510:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:29:58,0 nipype.interface INFO:
	 stderr 2024-10-28T16:29:58.000110:*+ WARNING: Set TR of output dataset to 1.0 s
241028-16:29:58,72 nipype.interface INFO:
	 stderr 2024-10-28T16:29:58.072198:++ elapsed time = 0.1 s
241028-16:29:58,742 nipype.interface INFO:
	 stderr 2024-10-28T16:29:58.742301:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:29:58,745 nipype.interface INFO:
	 stderr 2024-10-28T16:29:58.745066:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:29:58,777 nipype.interface INFO:
	 stderr 2024-10-28T16:29:58.777448:++ elapsed time = 0.0 s
sbm-nd-h


['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-

100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


241028-16:30:13,564 nipype.interface INFO:
	 stderr 2024-10-28T16:30:13.564567:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:30:13,571 nipype.interface INFO:
	 stderr 2024-10-28T16:30:13.571391:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:30:13,574 nipype.interface INFO:
	 stderr 2024-10-28T16:30:13.574002:*+ WARNING: Set TR of output dataset to 1.0 s
241028-16:30:13,649 nipype.interface INFO:
	 stderr 2024-10-28T16:30:13.649479:++ elapsed time = 0.1 s
241028-16:30:14,343 nipype.interface INFO:
	 stderr 2024-10-28T16:30:14.343310:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241028-16:30:14,346 nipype.interface INFO:
	 stderr 2024-10-28T16:30:14.346309:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241028-16:30:14,379 nipype.interface INFO:
	 stderr 2024-10-28T16:30:14.379587:++ elapsed time = 0.0 s
241028-16:30:15,479 nipype.interface INFO:
	 stderr 2024-10-28T16:30:15.479483:++

In [12]:
profiles_df

,sub,level,roi,profile,strength
0,SLC01,0,0,"[0.976, 0.0, 0.0, 0.0, 0.0, 0.024, 0.0, 0.0, 0...",0.357
1,SLC01,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.285
2,SLC01,0,0,"[0.0, 0.0, 0.003, 0.974, 0.0, 0.023, 0.0, 0.0,...",0.258
3,SLC01,0,0,"[0.0, 0.0, 0.0, 0.131, 0.0, 0.869, 0.0, 0.0, 0...",0.098
4,SLC01,0,0,"[0.2, 0.0, 0.0, 0.4, 0.0, 0.4, 0.0, 0.0, 0.0, ...",0.000
...,...,...,...,...,...
19387,SLC10,3,157,"[0.972, 0.004, 0.015, 0.006, 0.001, 0.001, 0.001]",0.999
19388,SLC10,3,158,"[0.972, 0.004, 0.015, 0.006, 0.001, 0.001, 0.001]",0.999
19389,SLC10,3,159,"[0.971, 0.004, 0.015, 0.006, 0.001, 0.001, 0.001]",0.999
19390,SLC10,3,160,"[0.972, 0.004, 0.015, 0.006, 0.001, 0.001, 0.001]",0.999
